In [1]:
library(tidyverse)
#library(Hmisc)
library(boot)
library(here)
library(geepack)
library(readxl)
library(haven)
library(lavaan)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
here() starts at /home/arinbasu/hlth_403
This is lavaan 0.6-5
lavaan is BETA software! Please report any bugs.


In [21]:
#library(remotes)
#install.packages("dagitty")
library(lavaan)

This is lavaan 0.6-5
lavaan is BETA software! Please report any bugs.


## Purpose and overview
The goal of this paper is to write about how to assess causal inference using directed acyclic graphs and counterfactual theories of causation. In epidemiology and health sciences, if X is a cause of Y, then:

- We must first show that X is statistically significantly associated with Y and meets the criteria of internal validity
- We must demonstrate that X and Y association is NOT one of chance alone
- We must demonstrate that X and Y association cannot be explained by biases - selection bias, response bias, measurement bias
- We must demonstrate that X and Y association cannot be explained by confounding variables

After we have done these, we can assess the "weight" of evidence that X and Y are causally related by considering the nine criteria that Sir Austin Bradford Hill proposed (He called them "considerations" rather than criteria). The three main criteria are:

- Strength of association
- Temporality (cause must always precede effect)
- Dose-response gradient (or biological gradient)

## Causal inference based on DAGs
- DAGs are directed acyclic graphs
- Derives from path tracing rules of Sewall Wright
- If X and Y are connected using a system of paths that traverse between the two, then:
- We can start a path in any direction and move along that same direction traversing the network till we reach Y
- The path can contain only ONE covariance path (double edged path)
- The path can pass through one variable only once
- If there are n valid paths connecting X and Y such that each path i has coefficient of p_i, then:
- Covariance(X,Y) = p_1 * p_2 * .... * p_n or
- Covariance(X, Y) = $\Pi$p_i

## Where DAGs in a causal diagram differs from Sewall Wright's Path
- There are no two way arrows 
- Arrows flow only in one direction
- The parent and child nodes are dependent otherwise the nodes are independent
- All nodes must be present in the graph and paths specified

## Three kinds of paths
- All backdoor paths must remain closed
- Backdoor paths are open if they have confounding variables in them (measured or unmeasured)
- Backdoor paths are open if they have colliders in them that are conditioned on or controlled for
- So, condition confounding variables in open backdoor paths, but do not condition on colliders in closed backdoor paths
- Colliders are those where two arrows converge, e.g.,A -> B <- C
- Typically, in epidemiology for instance, colliders are those where an exposure or an intervention and an outcome share the same variables

## Mediators, Confounders, and colliders
- A -> M -> Y (A is the exposure or the intervention, Y is the outcome)
- M is  a mediator as it mediates the connection between A and Y
- L -> A; L -> Y; A -> Y; here L is a confounder
- A -> C <- Y ; here C is a collider
- Judea Pearl has named "Chains", "Forks", and "colliders"


## What happens?
- Confounding variables must be controlled for
- Controlling on colliders lead to selection bias

## Counterfactual theory
- A causes Y 
- Imagine A is binary, and can take 2 values 1, and 0
- Imagine Y is binary, takes 2 values 1, and 0

## Counterfactuals
- We will state any value "a" as a counterfactual to a value of A, if:
- Say we OBSERVE Y = 1 when A = 1
- What if A were to be 0, what would be the value of Y?
- "What if A were to be 0 now that A = 1" is A's counterfactual "a"
- Here, the corresponding value of Y under A's counterfactual is Y(a=0)

## Definition of causality
- P(Y = 1 | A = 1) is the likelihood of Y = 1 GIVEN A = 1
- P(Y = 1 | A = 0) is the likelihood of Y = 1 GIVEN A = 0 or controlled condition or condition under comparison
- Then Association Risk Ratio = P(Y = 1 | A = 1) / P(Y = 1 | A = 0)
- The causal risk ratio = P(Y(a = 1) = 1 / P(Y(a = 0) = 1)
- If the causal risk ratio == association risk ratio, then:
- Association is Causation, not otherwise.

## How do we find the counterfactuality?
- We weight the individuals with contrasting conditions
- We assess their associations
- Three types of weights: 
- Inverse probability weights
- Standardised weights
- Weights using instrumental variables

## Codes are given below
Use [NHEFS data](https://wwwn.cdc.gov/nchs/nhanes/nhefs/Default.aspx)

In [10]:
67.5 + 78.75

[1] 146.25

In [2]:
library(here)

In [3]:
#install.packages("here")

In [10]:
library(here)
library(readxl)
library(haven)

In [11]:
# We will use the NHEFS data to study
# whether smoking cessation leads to or causes weight gain

nhefs <- read_excel(here("NHEFS.xls"))
nhefs$cens <- ifelse(is.na(nhefs$wt82), 1, 0)

In [27]:
nhefs %>% head() # what the data looks like

seqn,qsmk,death,yrdth,modth,dadth,sbp,dbp,sex,age,⋯,pregnancies,cholesterol,hightax82,price71,price82,tax71,tax82,price71_82,tax71_82,cens
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
233,0,0,NA,NA,NA,175,96,0,42,⋯,NA,197,0,2.183594,1.739990,1.1022949,0.4619751,0.44378662,0.6403809,0
235,0,0,NA,NA,NA,123,80,0,36,⋯,NA,301,0,2.346680,1.797363,1.3649902,0.5718994,0.54931641,0.7929688,0
244,0,0,NA,NA,NA,115,75,1,56,⋯,2,157,0,1.569580,1.513428,0.5512695,0.2309875,0.05619812,0.3202515,0
245,0,1,85,2,14,148,78,0,68,⋯,NA,174,0,1.506592,1.451904,0.5249023,0.2199707,0.05479431,0.3049927,0
252,0,0,NA,NA,NA,118,77,0,40,⋯,NA,216,0,2.346680,1.797363,1.3649902,0.5718994,0.54931641,0.7929688,0
257,0,0,NA,NA,NA,141,83,1,43,⋯,1,212,1,2.209961,2.025879,1.1547852,0.7479248,0.18408203,0.4069824,0


In [13]:
# remove missing values from NHEFS
nhefs.nmv <-
  nhefs[which(!is.na(nhefs$wt82)),] 


In [14]:
# regress weight change on quitting smoking status, 
lm(wt82_71 ~ qsmk, data = nhefs.nmv)


Call:
lm(formula = wt82_71 ~ qsmk, data = nhefs.nmv)

Coefficients:
(Intercept)         qsmk  
      1.984        2.541  


In [49]:
# qsmk = quitters (0 = did not quit, 1 = quit)
smokers <- nhefs.nmv %>%
 group_by(qsmk) %>%
 summarise(n = n(),
          age_yrs = mean(age))
gender <- nhefs %>%
 count(sex)
smokers
gender$n / sum(gender$n)

qsmk,n,age_yrs
<dbl>,<int>,<dbl>
0,1163,42.78848
1,403,46.17370


[1] 0.490485 0.509515

In [15]:
# fitting the ip weight
# they fit a logistic regression on the exposure variable/intervention variable
# with all potential confounders
fit <- glm(
  qsmk ~ sex + race + age + I(age ^ 2) +
    as.factor(education) + smokeintensity +
    I(smokeintensity ^ 2) + smokeyrs + I(smokeyrs ^ 2) +
    as.factor(exercise) + as.factor(active) + wt71 + I(wt71 ^ 2),
  family = binomial(),
  data = nhefs.nmv
)
summary(fit)


Call:
glm(formula = qsmk ~ sex + race + age + I(age^2) + as.factor(education) + 
    smokeintensity + I(smokeintensity^2) + smokeyrs + I(smokeyrs^2) + 
    as.factor(exercise) + as.factor(active) + wt71 + I(wt71^2), 
    family = binomial(), data = nhefs.nmv)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5127  -0.7907  -0.6387   0.9832   2.3729  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -2.2425191  1.3808360  -1.624 0.104369    
sex                   -0.5274782  0.1540496  -3.424 0.000617 ***
race                  -0.8392636  0.2100665  -3.995 6.46e-05 ***
age                    0.1212052  0.0512663   2.364 0.018068 *  
I(age^2)              -0.0008246  0.0005361  -1.538 0.124039    
as.factor(education)2 -0.0287755  0.1983506  -0.145 0.884653    
as.factor(education)3  0.0864318  0.1780850   0.485 0.627435    
as.factor(education)4  0.0636010  0.2732108   0.233 0.815924    
as.factor(education)5  0.

In [51]:
#install.packages("geepack")

Installing package into ‘/home/arinbasu/R/x86_64-conda_cos6-linux-gnu-library/3.5’
(as ‘lib’ is unspecified)


In [16]:
# We are creating the simple weights here
# What is the probability of observation that other variables predict quitting
p.qsmk.obs <-
  ifelse(nhefs.nmv$qsmk == 0,
         1 - predict(fit, type = "response"),
         predict(fit, type = "response"))

# we create a weight variable w by using inverse probability of the weights

nhefs.nmv$w <- 1 / p.qsmk.obs
summary(nhefs.nmv$w)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.054   1.230   1.373   1.996   1.990  16.700 

In [17]:
# Now we regress using general estimating equation model
# need "geepack" for this

library("geepack")
msm.w <- geeglm(
  wt82_71 ~ qsmk,
  data = nhefs.nmv,
  weights = w,
  id = seqn,
  corstr = "independence"
)
summary(msm.w)

## this tells us that the true CAUSAL weight gain is something like 3.44 Kg on an average


Call:
geeglm(formula = wt82_71 ~ qsmk, data = nhefs.nmv, weights = w, 
    id = seqn, corstr = "independence")

 Coefficients:
            Estimate Std.err  Wald Pr(>|W|)    
(Intercept)   1.7800  0.2247 62.73 2.33e-15 ***
qsmk          3.4405  0.5255 42.87 5.86e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation structure = independence 
Estimated Scale Parameters:

            Estimate Std.err
(Intercept)    65.06   4.221
Number of clusters:   1566  Maximum cluster size: 1 

In [54]:
3.44 - (1.96 * 0.53)

[1] 2.401

In [55]:
3.44 + (1.96 * 0.53)

[1] 4.479

In [ ]:
# 2.41 - 4.48 the 95% CI

In [38]:
## Instrumental variables
# for simplicity, ignore subjects with missing outcome or missing instrument
nhefs.iv <- nhefs[which(!is.na(nhefs$wt82) & !is.na(nhefs$price82)),]
nhefs.iv$highprice <- ifelse(nhefs.iv$price82>=1.5, 1, 0)

table(nhefs.iv$highprice, nhefs.iv$qsmk)

   
       0    1
  0   33    8
  1 1065  370

In [39]:
t.test(wt82_71 ~ highprice, data=nhefs.iv)


	Welch Two Sample t-test

data:  wt82_71 by highprice
t = -0.1, df = 42, p-value = 0.9
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -3.131  2.830
sample estimates:
mean in group 0 mean in group 1 
          2.536           2.686 


In [42]:
t2 <- '
wt82_71 ~ qsmk
qsmk ~ highprice
wt82_71 ~~ highprice
'

res2 <- lavaan::sem(model = t2, data = nhefs.iv)
summary(res2)

Warning message in lav_data_full(data = data, group = group, cluster = cluster, :
“lavaan WARNING: some observed variances are (at least) a factor 1000 times larger than others; use varTable(fit) to investigate”

lavaan 0.6-5 ended normally after 32 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of free parameters                          6
                                                      
  Number of observations                          1476
                                                      
Model Test User Model:
                                                      
  Test statistic                                 0.000
  Degrees of freedom                                 0

Parameter Estimates:

  Information                                 Expected
  Information saturated (h1) model          Structured
  Standard errors                             Standard

Regressions:
                   Estimate  Std.Err  z-value  P(>|z|)
  wt82_71 ~                                           
    qsmk              2.666    0.468    5.695    0.000
  qsmk ~                                              
    highpri

In [ ]:
# model1 <- tsls(wt82_71 ~ qsmk, ~ highprice, data = nhefs.iv)

In [43]:
t3 <- '
wt82_71 ~ qsmk
qsmk ~ highprice
'

res3 <- lavaan::sem(model = t3, data = nhefs.iv)
summary(res3)

lavaan 0.6-5 ended normally after 29 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of free parameters                          4
                                                      
  Number of observations                          1476
                                                      
Model Test User Model:
                                                      
  Test statistic                                 0.000
  Degrees of freedom                                 1
  P-value (Chi-square)                           0.989

Parameter Estimates:

  Information                                 Expected
  Information saturated (h1) model          Structured
  Standard errors                             Standard

Regressions:
                   Estimate  Std.Err  z-value  P(>|z|)
  wt82_71 ~                                           
    qsmk              2.666    0.468    5.696    0.000
  qsmk ~   